## PySpark ML Example
## Predict salary based on age and experience

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('MLExample').getOrCreate()

In [3]:
## read dataset
df=spark.read.csv('test6.csv',header=True,inferSchema=True)

In [4]:
df.show()

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|Giuseppe| 31|        10| 30000|
|   Mario| 30|         8| 25000|
|   Fabio| 29|         4| 20000|
|    Luca| 24|         3| 20000|
| Antonio| 21|         1| 15000|
|Federico| 23|         2| 18000|
+--------+---+----------+------+



In [5]:
## group independent features (in this case age and exp)
## use VectorAssembler to group age and exp in a new column named "Independent Features"
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age","Experience"],outputCol="Independent Features")

In [7]:
## add new features
output=featureassembler.transform(df)
output.show()

+--------+---+----------+------+--------------------+
|    Name|Age|Experience|Salary|Independent Features|
+--------+---+----------+------+--------------------+
|Giuseppe| 31|        10| 30000|         [31.0,10.0]|
|   Mario| 30|         8| 25000|          [30.0,8.0]|
|   Fabio| 29|         4| 20000|          [29.0,4.0]|
|    Luca| 24|         3| 20000|          [24.0,3.0]|
| Antonio| 21|         1| 15000|          [21.0,1.0]|
|Federico| 23|         2| 18000|          [23.0,2.0]|
+--------+---+----------+------+--------------------+



In [8]:
## select data that i'm interested in (independent features and salary)
final_data=output.select("Independent Features","Salary")
final_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [25]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=final_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [ ]:
### Prediction
pred_results=regressor.evaluate(test_data)
pred_results.predictions.show()